#### CARGA FATO PARQUET

In [2]:
dfvendas = spark.read.load('abfss://treinamento@adlstrn001.dfs.core.windows.net/silver/vendas/*.parquet', format='parquet')
dfvenda_itens = spark.read.load('abfss://treinamento@adlstrn001.dfs.core.windows.net/silver/venda_itens/*.parquet', format='parquet')

StatementMeta(poolspark1, 16, 2, Finished, Available)

#### JOIN DF VENDAS COM VENDAS ITENS

In [3]:
%%pyspark 
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql.functions import col
df_join_vendas = dfvendas.join(dfvenda_itens, dfvendas.NUM_VENDA == dfvenda_itens.NUM_VENDA)\
.select(dfvendas.NUM_VENDA,\
dfvendas.DATA_VENDA,\
dfvendas.ID_CLIENTE,\
dfvendas.STATUS,\
dfvendas.ID_VENDEDOR,\
dfvenda_itens.NUM_SEQ,\
dfvenda_itens.ID_PROD,\
dfvenda_itens.QTDE,\
dfvenda_itens.VAL_UNIT,\
dfvenda_itens.VAL_TOTAL,\
#,date_format('DATA_VENDA','yyyyMMDD').cast('Int').alias('DATA_VENDA_KEY')\
#,date_format('DATA_VENDA','yyyyMMdd').cast('String').alias('DATA_VENDA_CHAVE')\
date_format('DATA_VENDA','yyyyMMdd').cast('Int').alias('DATA_VENDA_CHAVE'),\
date_format('DATA_VENDA','yyyy').alias('ANO'),\
date_format('DATA_VENDA','MM').alias('MES')\
#date_format('DATA_VENDA','dd').alias('DIA'),\
)
#df_join_vendas.show(10)


StatementMeta(poolspark1, 16, 3, Finished, Available)

In [4]:
#verificando schema dp dataframe
df_join_vendas.schema

StatementMeta(poolspark1, 16, 4, Finished, Available)

StructType(List(StructField(NUM_VENDA,IntegerType,true),StructField(DATA_VENDA,TimestampType,true),StructField(ID_CLIENTE,IntegerType,true),StructField(STATUS,StringType,true),StructField(ID_VENDEDOR,IntegerType,true),StructField(NUM_SEQ,IntegerType,true),StructField(ID_PROD,IntegerType,true),StructField(QTDE,DecimalType(10,2),true),StructField(VAL_UNIT,DecimalType(10,2),true),StructField(VAL_TOTAL,DecimalType(10,2),true),StructField(DATA_VENDA_CHAVE,IntegerType,true),StructField(ANO,StringType,true),StructField(MES,StringType,true)))

#### FILTRAR VENDAS STATUS "N"

In [5]:
%%pyspark
join_filtrado = df_join_vendas.filter(df_join_vendas.STATUS=='N')
join_filtrado.show(10)

StatementMeta(poolspark1, 16, 5, Finished, Available)

+---------+-------------------+----------+------+-----------+-------+-------+-----+--------+---------+----------------+----+---+
|NUM_VENDA|         DATA_VENDA|ID_CLIENTE|STATUS|ID_VENDEDOR|NUM_SEQ|ID_PROD| QTDE|VAL_UNIT|VAL_TOTAL|DATA_VENDA_CHAVE| ANO|MES|
+---------+-------------------+----------+------+-----------+-------+-------+-----+--------+---------+----------------+----+---+
|        1|2017-01-02 00:00:00|         5|     N|          3|      1|      8|11.00| 3700.00| 40700.00|        20170102|2017| 01|
|        2|2017-01-03 00:00:00|         5|     N|          1|      1|      7|10.00| 2500.00| 25000.00|        20170103|2017| 01|
|        2|2017-01-03 00:00:00|         5|     N|          1|      2|      4|15.00|  600.00|  9000.00|        20170103|2017| 01|
|        2|2017-01-03 00:00:00|         5|     N|          1|      3|     13| 7.00|   90.00|   630.00|        20170103|2017| 01|
|        2|2017-01-03 00:00:00|         5|     N|          1|      4|      3|11.00|  700.00|  770

#### SALVAR ARQUIVO PARQUER PARTICIONADO ANO/MES

In [6]:
%%pyspark
##Escreve e salva arquivo Parquet particionado por ano
join_filtrado.write.partitionBy('ANO','MES').mode('overwrite').parquet("/gold/fato_vendas")

StatementMeta(poolspark1, 16, 6, Finished, Available)

#### SALVAR COMO TABELA SPARK

In [34]:
%%pyspark
join_filtrado.write.mode("overwrite").saveAsTable("ldw.FATO_VENDA")

StatementMeta(poolspark1, 5, 30, Finished, Available)